# Install the libraries

In [1]:
%pip install requests beautifulsoup4 pandas selenium chromedriver_autoinstaller torch fake-useragent python-dotenv keybert openpyxl pymongo openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 835.7/835.7 kB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 24.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: /Users/kuldeep/Documents/Projects/Jupyter Notebook/linkedinJobScrapper/seleniumenv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Initialize the driver

In [2]:
from selenium import webdriver
import chromedriver_autoinstaller
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)
print("Driver initialized successfully")

Driver initialized successfully


# Functions to login into the linkedin

In [3]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException;
import logging
def try_login(email, password):
    # Step 1: Navigate to LinkedIn login page
    driver.get("https://www.linkedin.com/login")
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "username"))
    )
    # Step 2: Perform login
    email_input = driver.find_element(By.ID, "username")
    password_input = driver.find_element(By.ID, "password")
    email_input.send_keys(email)
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN)
    sleep(3)
    print("There might be a captcha.")
        
    # options = Options()

def enter_verification_code(verificationCode):
    try:
        verificationCodeInput = driver.find_element(By.ID, "input__email_verification_pin")
        verificationCodeInput.send_keys(verificationCode)
        print("Verification code box found")
        verificationCodeInput.send_keys(Keys.RETURN)
    except NoSuchElementException:
        print("No need to enter verification code")
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "global-nav-search"))
        )
        logging.info("Login successful.")
    except TimeoutException:
        logging.error("Login failed. Please check your credentials.")
        driver.quit()
    

### You may need to fill catpcha manually to login

In [4]:
try_login("aakhyachaudhary2000@gmail.com", "Linked@1234")

There might be a captcha.


## Enter verification code(if recieved)

In [4]:
#Pass verification code into the function 
enter_verification_code("Enter code here")

No need to enter verification code


# Utility function to scroll jobs to load them all

In [18]:
def is_scrollable(element):
    return driver.execute_script("return arguments[0].scrollHeight > arguments[0].clientHeight", element)

def scrollUntilAllJobsLoad(scrollableElm):
    if is_scrollable(scrollableElm):
        last_height = driver.execute_script("return arguments[0].scrollHeight", scrollableElm)
        while True:
            # Scroll to the bottom of the child element
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollableElm)
            # Wait for new content to load
            sleep(2)

            # Check the new scroll height
            new_height = driver.execute_script("return arguments[0].scrollHeight", scrollableElm)
            # Break if no new content is loaded
            if new_height == last_height:
                break
            last_height = new_height
    
        # print("\tFinished scrolling the element.")
    else:
        print("\tPage is not scrollable.")

## Utility function to populate the dataframe with the jobs data

In [20]:
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
            
def populate_jobs_data(jobs, nbOfJobs):
    job_data = {
        "Title": [],
        "Company": [],
        "Location": [],
        "Link": [],
        "Description": [],
    }
    
    totalJobs = len(jobs) if nbOfJobs > len(jobs) else nbOfJobs

    for idx, job in enumerate(jobs[: totalJobs]):
        try:
            ActionChains(driver).move_to_element(job).perform()  
            
            job.click()
            # visibility_of_element_located
            titleElement = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, ".job-view-layout h1"))  # Title
            )
            title = titleElement.text
            
            companyElement = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, "[data-view-name='job-details-about-company-name-link']"))  # Company
            )
            company = companyElement.text
            
            locationElement = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, ".job-details-jobs-unified-top-card__primary-description-container span"))  # Location
            )
            location = locationElement.text
            
            linkElement = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, ".job-view-layout h1 a"))  # Link
            )
            link = linkElement.get_attribute("href")
            
            descriptionElement = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.ID, "job-details"))  # Description
            )
            description = descriptionElement.text
            
            job_data["Company"].append(company)
            job_data["Title"].append(title)
            job_data["Location"].append(location)
            job_data["Link"].append(link)
            job_data["Description"].append(description)
            
        except (ElementClickInterceptedException, TimeoutException, NoSuchElementException, StaleElementReferenceException) as e:
            job_data["Company"].append("")
            job_data["Title"].append("")
            job_data["Location"].append("")
            job_data["Link"].append("")
            job_data["Description"].append("")
            logging.warning(f"Error extracting job {idx}: {e}")
            print(e)
    
    # print("\tPopulating data completed.")
    return pd.DataFrame(job_data)

In [21]:
def scrape_jobs_from_page(nbOfJobsToScrapeFromPage):
    jobsContainer = driver.find_element(By.CSS_SELECTOR, ".scaffold-layout__list > div")
    scrollUntilAllJobsLoad(jobsContainer)
    jobs = driver.find_elements(By.CSS_SELECTOR, ".scaffold-layout__list > div > ul > li")
    return populate_jobs_data(jobs, nbOfJobsToScrapeFromPage)

In [22]:
JOBS_PER_PAGE = 25

In [23]:
def move_to_next_page(page):
    try:
        print(f"Waiting to navigate to page {page}")
        wait = WebDriverWait(driver, 10)
        
        pagination_button = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, f'button[aria-label="Page {page}"]'))
        )
        pagination_button.click()
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".job-view-layout h1")))
        print(f"Navigated to {page} page") 
    except Exception as e:
        print(f"An error occurred while navigating to page {page}: {e}")

In [24]:
def scrape_linkedin_jobs(nbJobs, url = "https://www.linkedin.com/jobs/search?keywords=Data+Center&location=United+States"):
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".scaffold-layout__list"))
    )
    totalPagesToScrape = (nbJobs + JOBS_PER_PAGE - 1) // JOBS_PER_PAGE
    currentPage = 1;
    totalJobsDf = pd.DataFrame()
    while True:
        print(f"Scraping from page: {currentPage}")
        nbOfJobsToScrapeFromPage = (nbJobs % JOBS_PER_PAGE) if totalPagesToScrape == currentPage else JOBS_PER_PAGE
        currentPageJobs = pd.DataFrame()
        # print(f"\tNumber of jobs that need to be fetch: {nbOfJobsToScrapeFromPage}")
        currentPageJobs = scrape_jobs_from_page(nbOfJobsToScrapeFromPage)
        totalJobsDf = pd.concat([totalJobsDf, currentPageJobs], ignore_index=True)
        if currentPage >= totalPagesToScrape or totalJobsDf.shape[0] >= nbJobs or totalJobsDf.shape[0] >= 1000:
            break
        move_to_next_page(currentPage + 1)
        currentPage += 1
    return totalJobsDf

In [25]:
from enum import Enum

class ExperienceFilter(Enum):
    ALL = ""
    INTERNSHIP = "1"
    ENTRY_LEVEL = "2"
    ASSOCIATE = "3"
    MID_SENIOR_LEVEL = "4"
    DIRECTOR = "5"
    EXECUTIVE = "6"

class JobPostTime(Enum):
    ANY_TIME = ""
    PAST_24_HOURS = "2"
    PAST_WEEK = "3"
    PAST_MONTH = "4"

In [26]:
JOB_EXCEL_FILE_INDEX = 1

In [27]:
def build_url_and_scrape_jobs(numberOfJobsToScrape, searchQuery = "Data Center", location = "United States"):
    global JOB_EXCEL_FILE_INDEX
    searchQuery = searchQuery.replace(" ", "%20")
    location = location.replace(" ", "%20")
    jobBaseUrl = f"https://www.linkedin.com/jobs/search?keywords={searchQuery}&location={location}"
    jobsScraped = 0
    doesFoundAllJobs = False
    for experience in ExperienceFilter:
        for postTime in JobPostTime:
            url = jobBaseUrl + f"&f_E={experience.value}&f_TPR={postTime.value}"
            print(f"\nScrapping from: {url}")
            jobsDF = scrape_linkedin_jobs(numberOfJobsToScrape, url)
            jobsDF = jobsDF[~(jobsDF.eq('').all(axis=1))]
            jobsDF.to_excel(f"job_data_{JOB_EXCEL_FILE_INDEX}.xlsx", index=False)
            jobsScraped = jobsScraped + jobsDF.shape[0]
            JOB_EXCEL_FILE_INDEX = JOB_EXCEL_FILE_INDEX + 1
            if(jobsScraped >= numberOfJobsToScrape):
                print(f"Total jobs scraped: {jobsScraped}")
                return;
    print(f"You entered high number of jobs, we can scrape only {jobsScraped} number of jobs.")
    

# Start scrapping the jobs

In [28]:
searchQuery = "Data Center"
location = "United States"
numberOfJobsToScrape = 25000

In [29]:
build_url_and_scrape_jobs(numberOfJobsToScrape, searchQuery, location)


Scrapping from: https://www.linkedin.com/jobs/search?keywords=Data%20Center&location=United%20States&f_E=&f_TPR=
Scraping from page: 1
Waiting to navigate to page 2
Navigated to 2 page
Scraping from page: 2
Waiting to navigate to page 3
Navigated to 3 page
Scraping from page: 3
Waiting to navigate to page 4
Navigated to 4 page
Scraping from page: 4
Waiting to navigate to page 5
Navigated to 5 page
Scraping from page: 5
Waiting to navigate to page 6
Navigated to 6 page
Scraping from page: 6
Waiting to navigate to page 7
Navigated to 7 page
Scraping from page: 7
Waiting to navigate to page 8
Navigated to 8 page
Scraping from page: 8
Waiting to navigate to page 9
Navigated to 9 page
Scraping from page: 9
Waiting to navigate to page 10
Navigated to 10 page
Scraping from page: 10
Waiting to navigate to page 11
Navigated to 11 page
Scraping from page: 11
Waiting to navigate to page 12
Navigated to 12 page
Scraping from page: 12
Waiting to navigate to page 13
Navigated to 13 page
Scraping fr

Stacktrace:
0   chromedriver                        0x0000000102ba3184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x0000000102b9b9d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x0000000102608968 cxxbridge1$string$len + 89228
3   chromedriver                        0x000000010264cd50 cxxbridge1$string$len + 368756
4   chromedriver                        0x00000001026865b4 cxxbridge1$string$len + 604376
5   chromedriver                        0x0000000102641568 cxxbridge1$string$len + 321676
6   chromedriver                        0x00000001026421b8 cxxbridge1$string$len + 324828
7   chromedriver                        0x0000000102b6e9ac cxxbridge1$str$ptr + 3411716
8   chromedriver                        0x0000000102b71ccc cxxbridge1$str$ptr + 3424804
9   chromedriver                        0x0000000102b5586c cxxbridge1$str$ptr + 3308996
10  chromedriver                        0x0000000102b7258c cxxbridge1$str$ptr + 3427044
11  chromed

Message: 
Stacktrace:
0   chromedriver                        0x0000000102ba3184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x0000000102b9b9d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x0000000102608968 cxxbridge1$string$len + 89228
3   chromedriver                        0x000000010264cd50 cxxbridge1$string$len + 368756
4   chromedriver                        0x00000001026865b4 cxxbridge1$string$len + 604376
5   chromedriver                        0x0000000102641568 cxxbridge1$string$len + 321676
6   chromedriver                        0x00000001026421b8 cxxbridge1$string$len + 324828
7   chromedriver                        0x0000000102b6e9ac cxxbridge1$str$ptr + 3411716
8   chromedriver                        0x0000000102b71ccc cxxbridge1$str$ptr + 3424804
9   chromedriver                        0x0000000102b5586c cxxbridge1$str$ptr + 3308996
10  chromedriver                        0x0000000102b7258c cxxbridge1$str$ptr + 3427044
1

  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000102ba3184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x0000000102b9b9d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x0000000102608968 cxxbridge1$string$len + 89228
3   chromedriver                        0x000000010260d84c cxxbridge1$string$len + 109424
4   chromedriver                        0x000000010260f89c cxxbridge1$string$len + 117696
5   chromedriver                        0x000000010260f944 cxxbridge1$string$len + 117864
6   chromedriver                        0x0000000102647f14 cxxbridge1$string$len + 348728
7   chromedriver                        0x0000000102642ec8 cxxbridge1$string$len + 328172
8   chromedriver                        0x00000001026865b4 cxxbridge1$string$le

Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000102ba3184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x0000000102b9b9d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x0000000102608968 cxxbridge1$string$len + 89228
3   chromedriver                        0x000000010260d84c cxxbridge1$string$len + 109424
4   chromedriver                        0x000000010260f89c cxxbridge1$string$len + 117696
5   chromedriver                        0x000000010260f944 cxxbridge1$string$len + 117864
6   chromedriver                        0x0000000102647f14 cxxbridge1$string$len + 348728
7   chromedriver                        0x0000000102642ec8 cxxbridge1$string$len + 328172


  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000102ba3184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x0000000102b9b9d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x0000000102608968 cxxbridge1$string$len + 89228
3   chromedriver                        0x000000010260d84c cxxbridge1$string$len + 109424
4   chromedriver                        0x000000010260f89c cxxbridge1$string$len + 117696
5   chromedriver                        0x000000010260f944 cxxbridge1$string$len + 117864
6   chromedriver                        0x0000000102647f14 cxxbridge1$string$len + 348728
7   chromedriver                        0x0000000102642ec8 cxxbridge1$string$len + 328172
8   chromedriver                        0x00000001026865b4 cxxbridge1$string$le

Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000102ba3184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x0000000102b9b9d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x0000000102608968 cxxbridge1$string$len + 89228
3   chromedriver                        0x000000010260d84c cxxbridge1$string$len + 109424
4   chromedriver                        0x000000010260f89c cxxbridge1$string$len + 117696
5   chromedriver                        0x000000010260f944 cxxbridge1$string$len + 117864
6   chromedriver                        0x0000000102647f14 cxxbridge1$string$len + 348728
7   chromedriver                        0x0000000102642ec8 cxxbridge1$string$len + 328172


  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000102ba3184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x0000000102b9b9d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x0000000102608968 cxxbridge1$string$len + 89228
3   chromedriver                        0x000000010260d84c cxxbridge1$string$len + 109424
4   chromedriver                        0x000000010260f89c cxxbridge1$string$len + 117696
5   chromedriver                        0x000000010260f944 cxxbridge1$string$len + 117864
6   chromedriver                        0x0000000102647f14 cxxbridge1$string$len + 348728
7   chromedriver                        0x0000000102642ec8 cxxbridge1$string$len + 328172
8   chromedriver                        0x00000001026865b4 cxxbridge1$string$le

Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000102ba3184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x0000000102b9b9d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x0000000102608968 cxxbridge1$string$len + 89228
3   chromedriver                        0x000000010260d84c cxxbridge1$string$len + 109424
4   chromedriver                        0x000000010260f89c cxxbridge1$string$len + 117696
5   chromedriver                        0x000000010260f944 cxxbridge1$string$len + 117864
6   chromedriver                        0x0000000102647f14 cxxbridge1$string$len + 348728
7   chromedriver                        0x0000000102642ec8 cxxbridge1$string$len + 328172


KeyboardInterrupt: 

,Title,Company,Location,Link,Description
0,Plant Cyber Security Analyst,X-energy,"Rockville, MD",https://www.linkedin.com/jobs/view/4099647478/...,About the job\nX-energy LLC conducts a thoroug...
1,Senior Physical Security Systems Technician - ...,VetJobs,"San Jose, CA",https://www.linkedin.com/jobs/view/4095124819/...,About the job\nJob Description\n\nATTENTION MI...
2,CAD Drafter,V2D Convergence,"Austin, TX",https://www.linkedin.com/jobs/view/4103451104/...,About the job\nCompany Description\n\nMeade En...
3,Customer Service Analyst (Active SECRET Cleara...,Seneca Nation Group,"Denver, CO",https://www.linkedin.com/jobs/view/4083550577/...,About the job\nGreat Hill Solutions is part of...
4,Distributed Generation Demand Response Lead,Stream Data Centers,United States,https://www.linkedin.com/jobs/view/4075634185/...,"About the job\nFor 25 years, Stream Data Cente..."
...,...,...,...,...,...
270,"Manager or Senior Manager, Managed Services",KPMG US,"Portland, OR",https://www.linkedin.com/jobs/view/4104183428/...,"About the job\nAt KPMG, you can become an inte..."
271,Senior Network Security Engineer - MicroSegmen...,Mizuho,"Iselin, NJ",https://www.linkedin.com/jobs/view/4080339704/...,About the job\nJoin the Mizuho team as a Senio...
272,"Manager or Senior Manager, Managed Services",KPMG US,"Austin, TX",https://www.linkedin.com/jobs/view/4110331870/...,"About the job\nAt KPMG, you can become an inte..."
273,Fluid Systems Operations Engineer II,Blue Origin,"Seattle, WA",https://www.linkedin.com/jobs/view/4090986740/...,"About the job\nAt Blue Origin, we envision mil..."


### Save the jobs in Mongo DB

In [4]:
import pandas as pd
from pymongo import MongoClient

import glob
import os

folder_path = '.'
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

for file in excel_files:
    print(f"processing {file} file.")
    # Step 1: Read data from Excel
    file_path = os.path.join(folder_path, file)  # Replace with your file path
    df = pd.read_excel(file_path)
    # Step 2: Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')  # Use your MongoDB connection string
    db = client['local']  # Replace with your database name
    collection = db['linked_jobs']  # Replace with your collection name
    
    # Step 3: Convert DataFrame to Dictionary
    # Each row of the DataFrame will be converted into a dictionary
    data_dict = df.to_dict(orient='records')
    
    # Step 4: Insert Data into MongoDB
    collection.insert_many(data_dict)
    
    print(f"Jobs from {file} has been successfully inserted into MongoDB.")


excel
['./job_data_1.xlsx', './job_data_2.xlsx']
processing ./job_data_1.xlsx file.
dataframe = 
                                                 Title              Company  \
0                         Plant Cyber Security Analyst             X-energy   
1    Senior Physical Security Systems Technician - ...              VetJobs   
2                                          CAD Drafter      V2D Convergence   
3    Customer Service Analyst (Active SECRET Cleara...  Seneca Nation Group   
4          Distributed Generation Demand Response Lead  Stream Data Centers   
..                                                 ...                  ...   
995  Psychiatry Physician Job with OhioHealth in Ma...         PracticeLink   
996              Director of Professional Services, RN  Concierge Home Care   
997  Senior Manager of Infrastructure & Cloud Opera...    TransMedics, Inc.   
998      Senior Manager, Network Capacity and Strategy  Madronich Dr Robert   
999  Cardio-Thoracic Surgery Physi

# Close the driver

In [125]:
driver.quit()

div
class: ['EqJniyihVFUsazZbXesrlrNpmyhxhQ']
Found scrollable element. Scrolling...
div
class: ['EqJniyihVFUsazZbXesrlrNpmyhxhQ']
Finished scrolling the element.
Scrolling complete for all scrollable elements.
